In [11]:
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt


In [14]:

# 加载数据
file_path = 'q1/Attachment 1.xlsx'
data = pd.read_excel(file_path)
time = data['MJD(days)']  # 使用 MJD(days) 作为时间列
observed_arrival_time = data['PT-TT（s）']  # 使用 PT-TT（s） 作为观测到达时间

# 定义红噪声模型
def red_noise_model(t, P0, fc, q):
    """
    红噪声模型函数，用于拟合脉冲星计时噪声。
    t: 时间
    P0: 红噪声强度
    fc: 转角频率
    q: 频谱指数
    """
    f = np.fft.fftfreq(len(t))  # 计算频率
    noise_spectrum = P0 / (1 + (f / fc)**2)**(q / 2)
    return np.fft.ifft(noise_spectrum).real

# 初始参数
initial_params = [1.0, 0.1, 2.0]  # 假设的初始参数 [P0, fc, q]

# 数据拟合
try:
    popt, pcov = curve_fit(red_noise_model, time, observed_arrival_time, p0=initial_params)
    P0, fc, q = popt
    print(f'拟合参数：P0={P0:.4f}, fc={fc:.4f}, q={q:.4f}')
except Exception as e:
    print(f"拟合失败: {e}")

# 计算拟合结果
fitted_values = red_noise_model(time, *popt)

# 计算R²值
ss_res = np.sum((observed_arrival_time - fitted_values) ** 2)
ss_tot = np.sum((observed_arrival_time - np.mean(observed_arrival_time)) ** 2)
r_squared = 1 - (ss_res / ss_tot)
print(f'模型的R²值: {r_squared:.4f}')

# 结果可视化
plt.figure(figsize=(10, 6))
plt.plot(time, observed_arrival_time, label='Observed Arrival Time')
plt.plot(time, fitted_values, label='Fitted Model', linestyle='--')
plt.xlabel('Time (MJD)')
plt.ylabel('Arrival Time Difference (PT-TT)')
plt.title('Pulsar Timing Noise Model Fitting')
plt.legend()
plt.show()

# 检查模型拟合度是否达到95%以上
if r_squared >= 0.95:
    print("模型拟合度达到95%以上，符合要求。")
else:
    print("模型拟合度未达到95%，需要进一步优化模型。")


KeyError: 'MJD(days)'